# Covid vaccines used by country worldwide

As I was curious about the usage of the different vaccins worldwide, I decided to make a Notebook about it. As Gabriel Preda already made a dataset with the required data downloaded from Our World in Data, I am using his dataset.

## Table of contents
[**1. Loading and preparing data**](#1.-Loading-and-preparing-data)

[**2. Which vaccines are used where?**](#2.-Which-vaccines-are-used-where?)

[2.1 Worldwide usage maps by vaccin](#2.1-Worldwide-usage-maps-by-vaccin)

[2.2 Worldwide map plotting the number of vaccines used by country](#2.2-Worldwide-map-plotting-the-number-of-vaccines-used-by-country)

[2.3 Usage of vaccines in Europe by country](#2.3-Usage-of-vaccines-in-Europe-by-country)

[2.4 Usage of vaccines in Europe over time](#2.4-Usage-of-vaccines-in-Europe-over-time)

[**3. Overall vaccination rates**](#3.-Overall-vaccination-rates)


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import plotly.express as px
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

# 1. Loading and preparing data

In [ ]:
manu = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations_by_manufacturer.csv")

In [ ]:
manu.tail(3)

In [ ]:
manu['location'].unique()

In [ ]:
manu['date'] = pd.to_datetime(manu.date)

In [ ]:
manu.info()

In [ ]:
country = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
country.info()

In [ ]:
country.describe()

In [ ]:
country['country'].unique()

In [ ]:
len(country['country'].unique())

In [ ]:
country.info('south korea')

In [ ]:
country['date'] = pd.to_datetime(country.date)

In [ ]:
country.info()

The file with country vaccinations by manufacturer seems most promising as it holds detailed information the vaccinations given per vaccine per date per country.

In [ ]:
manu.head()

However, it turns out that this information is only available for a limited number of countries.

In [ ]:
manu.location.unique()

Although less ideal, it turns out that the country file holds a column that specifies all vaccins used in a country

In [ ]:
#temporarily set max max_colwidth to None
pd.set_option('display.max_colwidth', None)
country[['country', 'date', 'vaccines']].head()

Now, I want to keep the best-known vaccins (Pfizer, Moderna, AstraZeneca and Johnson&Johnson) as they are and group lesser-known brands into the countries where they are made.

In [ ]:
country['vaccines'].head(3)

In [ ]:
#strip text
country['vaccines'] = country['vaccines'].str.replace("/Beijing|/Wuhan|/HayatVax|/BioNTech|Oxford/", "", regex=True)

#replace with Chinese vaccins
country['vaccines'] = country['vaccines'].str.replace("Sinopharm|BBIBP-CorV|Sinovac|CanSino|RBD-Dimer", "Chinese", regex=True)

#replace with Russian vaccins
country['vaccines'] = country['vaccines'].str.replace("EpiVacCorona|Sputnik V", "Russian", regex=True)

#replace with Cuban vaccins
country['vaccines'] = country['vaccines'].str.replace("Soberana02|Abdala", "Cuban", regex=True)

In [ ]:
#replace some others
to_replace = {'Covaxin': 'Indian',
              'QazVac': 'Kazachstan'}
country['vaccines'] = country['vaccines'].replace(to_replace, regex=True)

In [ ]:
#making a list of all vaccins
vac_list = [x.split(", ") for x in country.vaccines.values]
vaccins = [item for elem in vac_list for item in elem]
vaccins = set(vaccins)
vaccins = list(vaccins)
vaccins

Now, I want to add a column with True/False for each vaccin, and only keep the row with the most recent information for each country.

In [ ]:
#adding a column with True/False for each vaccin
for vaccin in vaccins:
    country[vaccin] = np.where(country['vaccines'].str.contains(vaccin), True, False)

In [ ]:
country = country.sort_values(by = ['country', 'date'], ascending = [True, False])
country_latest = country.drop_duplicates(subset = "country", keep = "first")

In [ ]:
#head of selected columns only
country_latest.iloc[:, np.r_[0,12, 15:len(country_latest.columns)]].head()

In [ ]:
country_latest.head()

# 2. Which vaccines are used where?

## 2.1 Worldwide usage maps by vaccin

Below, I am making a function that allows me to specify which vaccin I want to display on a worldwide map and the color that I want to give to the countries using the vaccin.

In [ ]:
def plot_vaccin(color, vaccin):
    fig = px.choropleth(country_latest, locations="iso_code",
                        color=vaccin,
                        hover_name="country",
                        color_discrete_map={True: color, False: 'lightgrey'})

    layout = go.Layout(
        title=go.layout.Title(
            text= f"<b>Countries using {vaccin} vaccin</b>",
            x=0.5
        ),
        showlegend=False,
        font=dict(size=14),
        width = 750,
        height = 350,
        margin=dict(l=0,r=0,b=0,t=30)
    )

    fig.update_layout(layout)

    fig.show()

As we can see, Pfizer is used a lot in the Western world.

In [ ]:
plot_vaccin('red', 'Pfizer')

Moderna's footprint of countries is smaller as it is mostly the US, Canada and Europe.

In [ ]:
plot_vaccin('green', "Moderna")

AstraZeneca is used in a lot of countries including most African and South American countries. Some noticable countries missing here are the US, South Africa and some European countries such as Norway and Denmark.

In [ ]:
plot_vaccin('brown', "AstraZeneca")

Johnson & Johnson's footprint is smaller. Noticable is that South Africa uses Johnson & Johnson while not using AstraZenca (the opposite of most African countries).

In [ ]:
plot_vaccin('orange', "Johnson&Johnson")

Chinese brands are used in a lot of Asian countries and also almost all South American countries use at least some Chinese vaccins.

In [ ]:
plot_vaccin('blue', "Chinese")

Besides Russia itself, there is a significant number of countries also using Russian vaccins (I suppose mostly Sputnik as I had not heard about the other Russian vaccin yet).

In [ ]:
plot_vaccin('yellow', "Russian")

The Indian vaccin (Covaxin) is used in India and a few other countries.

In [ ]:
plot_vaccin('magenta', "Indian")

I had not heard about the Cuban vaccins yet, but it turns out that those are only used in Cuba itself.

In [ ]:
plot_vaccin('goldenrod', "Cuban")

The same things applies for the vaccin made in Kazachstan: it is only used in the country itself.

In [ ]:
plot_vaccin('darkblue', "Kazachstan")

## 2.2 Worldwide map plotting the number of vaccines used by country

As you can see, many countries use only one or two different vaccines.

In [ ]:
country_latest['Vaccins_used']= country_latest.iloc[:, -9:].sum(axis=1)
country_latest.Vaccins_used.value_counts()

Noticable is that only Libya and Hungary use more than 4 different vaccines.

In [ ]:
fig = px.choropleth(country_latest, locations="iso_code",
                    color='Vaccins_used',
                    hover_data= ["country", "vaccines"])

layout = go.Layout(
    title=go.layout.Title(
        text= f"<b>Number of different vaccines used by country</b>",
        x=0.5
    ),
    font=dict(size=14),
    width = 750,
    height = 350,
    margin=dict(l=0,r=0,b=0,t=30)
)

fig.update_layout(layout)

fig.show()

## 2.3 Usage of vaccines in Europe by country

As mentioned before, the file by manufacturer does not hold all countries. However, it does contain detailed info on most European countries. I decided to get rid of the 3 countries outside Europe and to dig deeper anyway.

In [ ]:
non_european = ['Chile', 'United States', 'Uruguay']
manu = manu.query('location not in @non_european')
manu.location.nunique()

By just checking some info of my own country, I see that the numbers are clearly cumulatief.

In [ ]:
manu[manu.location == "south korea"]

In [ ]:
manu[manu.location == "Netherlands"].tail(8)

In [ ]:
manu_totals = manu.copy()

#keeping only the latest info
manu_totals = manu_totals.sort_values(by = ['location', 'date', 'vaccine'], ascending = [True, False, True])
manu_totals = manu_totals.drop(columns = "date")
manu_totals = manu_totals.drop_duplicates(subset = ['location', 'vaccine'], keep = "first")

manu_totals = manu_totals.pivot(index = ['location'], columns = 'vaccine', values = 'total_vaccinations')
manu_totals = manu_totals.fillna(0)
manu_totals.iloc[:, -6:] = manu_totals.iloc[:, -6:].astype(int)

#convert to percent
cols = list(manu_totals.columns)
manu_totals[cols] = manu_totals[cols].div(manu_totals[cols].sum(axis=1), axis=0).multiply(100)
manu_totals = manu_totals.round(1)
                                
#adding ISO codes from my own dataset https://www.kaggle.com/erikbruin/countries-of-the-world-iso-codes-and-population
countries_iso = pd.read_csv("../input/countries-of-the-world-iso-codes-and-population/country_codes_2020.csv")
countries_iso = countries_iso.rename(columns = {'name': 'location', 'cca3': 'iso_code'})

manu_totals = manu_totals.reset_index()
manu_totals = manu_totals.merge(countries_iso[['location', 'iso_code']], on = 'location', how = "left")

However, this time I am interested to see what percentage of vaccines uses was mRNA (Pfizer and Moderna).

In [ ]:
manu_totals['mRNA'] = manu_totals['Moderna'] + manu_totals['Pfizer/BioNTech']
manu_totals.head()

In [ ]:
fig = px.choropleth(manu_totals, locations="iso_code",
                    color='mRNA',
                    hover_data= ["location"])

layout = go.Layout(
    title=go.layout.Title(
        text= f"<b>Percent mRNA vaccines used by country</b>",
        x=0.5
    ),
    font=dict(size=14),
    autosize=False,
    width = 800,
    height = 600,
    margin=dict(l=0,r=0,b=0,t=30)
)
fig.update_geos(scope="europe")
fig.update_layout(layout)

fig.show()


We see that especially Hungary uses a lot less mRNA. This is because Hungary is the only European country that also uses Sputnik and Sinopharm.

In [ ]:
manu_totals[manu_totals.location == "Hungary"]

## 2.4 Usage of vaccines in Europe over time


In [ ]:
manu = manu.sort_values(by=['location', 'vaccine', 'date'], ascending = [True, True, False])
manu = manu.reset_index(drop=True)

manu['new'] = 0
for i in range(len(manu)-1):
    if ((manu.loc[i+1, 'vaccine'] == manu.loc[i, 'vaccine']) & (manu.loc[i+1, 'location'] == manu.loc[i, 'location'])):
        manu.loc[i, 'new'] = (manu.loc[i, 'total_vaccinations'] - manu.loc[i+1, 'total_vaccinations'])
    else:
        manu.loc[i, 'new'] = manu.loc[i, 'total_vaccinations']
        
manu.head()

In [ ]:
new_vaccines = manu[['date', 'vaccine', 'new']]

df = new_vaccines[['date', 'vaccine']]
df.drop_duplicates(inplace = True)
df = df.sort_values(by = ['vaccine', 'date'], ascending = [True, False]).reset_index(drop=True)
df['before'] = 0

for i,row in df.iterrows():
    total_before = new_vaccines[((new_vaccines.vaccine == row.vaccine) & (new_vaccines.date <= row.date))]['new'].sum()
    df.loc[i, 'before'] = total_before
    
df = df.pivot(index= 'date', columns = 'vaccine', values = 'before')
df = df[['Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech']] #getting rid of Sputnik and Sinopharm
df.head()

As you can see below, Europe is increasingly relying on Pfizer. While Pfizer was used the most since the start of the vaccinations, the trend is also upward. The amounts used of Moderna and AstraZeneca are also significant. Johnson& Johnson is not used a lot yet.

In [ ]:
df.iplot(mode = 'lines', xTitle = 'Date', yTitle = 'Total vaccinations', title = 'Cumulative vaccinations in Europe by vaccine')

# 3. Overall vaccination rates

Next version ;-)

**To be continued: Please stay tuned!**